In [ ]:
# Este notebook contiene código para limpiar y procesar un dataset de seguros.
# Se documentarán las diferentes secciones y funciones para una mejor comprensión.

# Si no se está usando un ambiente virtual, descomentar y ejecutar las siguientes líneas
# Instalar la librería numpy si no está instalada
# !pip install numpy
# Instalar la librería pandas si no está instalada
# !pip install pandas
# Instalar la librería ucimlrepo si no está instalada (parece no usarse en el código actual, pero se mantiene por si acaso)
# !pip install ucimlrepo
# Instalar la librería seaborn si no está instalada (parece no usarse en el código actual, pero se mantiene por si acaso)
#!pip install seaborn

In [37]:
# Importar el módulo sys para acceder a funcionalidades del sistema
import sys
# Imprimir la ruta del ejecutable de Python que se está utilizando
print(sys.executable)

c:\Users\chile\anaconda3\envs\devstack\python.exe


In [38]:
# Cargando las librerías necesarias para el procesamiento de datos y visualización

# Módulo para interactuar con el sistema operativo
import os
# Módulo para operaciones con expresiones regulares
import re
# Librería para operaciones numéricas y arrays
import numpy as np
# Librería para manipulación y análisis de datos (DataFrames)
import pandas as pd
# Módulo para crear visualizaciones estáticas, interactivas y animadas en Python
import matplotlib.pyplot as plt
# Tipo de dato para columnas categóricas en pandas
from pandas.api.types import CategoricalDtype
# *Variable para controlar si se ignoran las advertencias
ignore_warnings = True
# *Librería para crear gráficos estadísticos atractivos
import seaborn as sns
# Módulo para trabajar con datos en formato JSON
import json
# *Función para obtener datasets del repositorio UCI
#from ucimlrepo import fetch_ucirepo
# *Módulo para trabajar con archivos YAML
import yaml
# Clase para trabajar con rutas de archivos de forma orientada a objetos
from pathlib import Path

In [41]:
# Ruta que estás usando para abrir el YAML
yaml_path = (Path.cwd().parent / "config" / "params.yaml").resolve()
print("Intentando abrir:", yaml_path)

# Muestra el contenido real del archivo que se está leyendo
text = yaml_path.read_text(encoding="utf-8")
print("\n--- CONTENIDO DEL YAML LEÍDO ---")
print(text)
print("--------------------------------\n")

cfg = yaml.safe_load(text)
#print("Claves de nivel 1:", list(cfg.keys()))
print("Claves en paths:", list(cfg.get("paths", {}).keys()))

# Validaciones amistosas
required = {"data_dir", "raw_dir", "clean_dir"}
missing = required - set(cfg.get("paths", {}).keys())
assert not missing, f"Faltan claves en paths: {missing}"


Intentando abrir: C:\Users\chile\projects\Mlops_proyect-main\config\params.yaml

--- CONTENIDO DEL YAML LEÍDO ---

--------------------------------



AttributeError: 'NoneType' object has no attribute 'get'

In [42]:
yaml_text = """paths:
  raw: data/enriched/insurance_company_enriched.csv
  interim: data/interim/data_clean.parquet
  processed: data/processed/data_model.parquet
  report_html: reports/eda_clean.html
  report_css: reports/assets/css/report.css
  root: .
  data_dir: data
  raw_dir: data/raw
  clean_dir: data/clean_data
"""

yaml_path = (Path.cwd().parent / "config" / "params.yaml").resolve()
yaml_path.write_text(yaml_text, encoding="utf-8")

print("Escribí:", yaml_path)
print("Tamaño (bytes):", yaml_path.stat().st_size)
print("Contenido guardado:\n")
print(yaml_path.read_text(encoding="utf-8"))

Escribí: C:\Users\chile\projects\Mlops_proyect-main\config\params.yaml
Tamaño (bytes): 317
Contenido guardado:

paths:
  raw: data/enriched/insurance_company_enriched.csv
  interim: data/interim/data_clean.parquet
  processed: data/processed/data_model.parquet
  report_html: reports/eda_clean.html
  report_css: reports/assets/css/report.css
  root: .
  data_dir: data
  raw_dir: data/raw
  clean_dir: data/clean_data



In [43]:
yaml_path = (Path.cwd().parent / "config" / "params.yaml").resolve()
print("Leyendo:", yaml_path)

with open(yaml_path, "r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)

print("Claves de nivel 1:", list(cfg.keys()))
print("Claves dentro de 'paths':", list(cfg["paths"].keys()))

Leyendo: C:\Users\chile\projects\Mlops_proyect-main\config\params.yaml
Claves de nivel 1: ['paths']
Claves dentro de 'paths': ['raw', 'interim', 'processed', 'report_html', 'report_css', 'root', 'data_dir', 'raw_dir', 'clean_dir']


In [ ]:
# Mostrar exactamente qué archivo se está abriendo
yaml_path = (Path.cwd().parent / "config" / "params.yaml").resolve()
print("Archivo YAML que se está abriendo:")
print(yaml_path)
print("\n¿Existe realmente ese archivo?:", yaml_path.exists())

In [ ]:
# Detectar raíz del proyecto (subiendo desde notebooks/)
PROJECT_ROOT = Path.cwd().parent

# Extraer rutas
DATA_DIR  = PROJECT_ROOT / cfg["paths"]["data_dir"]
RAW_DIR   = PROJECT_ROOT / cfg["paths"]["raw_dir"]
CLEAN_DIR = PROJECT_ROOT / cfg["paths"]["clean_dir"]

# Archivos
DICT_TXT   = PROJECT_ROOT / cfg["files"]["dictionary"]
CSV_PATH   = PROJECT_ROOT / cfg["files"]["csv_input"]
OUT_CSV    = PROJECT_ROOT / cfg["files"]["csv_output"]
OUT_REPORT = PROJECT_ROOT / cfg["files"]["report_output"]

# Crear carpeta de salida si no existe
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("CSV_PATH existe:", CSV_PATH.exists())
print("Salida limpia:", OUT_CSV)


In [ ]:
# Detectar la raíz del proyecto (un nivel arriba del notebook)
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

# Rutas basadas en la raíz real del proyecto
DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR  = DATA_DIR / "raw"
CLEAN_DIR = DATA_DIR / "clean_data"

# Archivos de entrada y salida
DICT_TXT   = RAW_DIR / "dictionary.txt"
CSV_PATH   = RAW_DIR / "insurance_company_modified.csv"
OUT_DIR    = CLEAN_DIR
OUT_CSV    = OUT_DIR / "coil2000_clean.csv"
OUT_REPORT = OUT_DIR / "coil2000_dtypes_report.csv"

# Crear el directorio de salida si no existe
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Mostrar verificación
print("PROJECT_ROOT:", PROJECT_ROOT)
print("DICT_TXT exists:", DICT_TXT.exists())
print("CSV_PATH exists:", CSV_PATH.exists())
print("OUT_DIR:", OUT_DIR)

In [ ]:
# Definir el número esperado de columnas en el dataset
N_COLS = 86
# Definir un conjunto con los nombres de las columnas que deben ser tratadas como categóricas
CAT_COLS = {"MOSTYPE", "MOSHOOFD", "MKOOPKLA"}

In [ ]:
# --- Configuración mínima del script ---
# Definir la ruta al archivo del diccionario de datos
#DICT_TXT = Path(r"./data/raw/dictionary.txt")
# Definir la ruta al archivo CSV original modificado del dataset de la compañía de seguros
#CSV_PATH = Path(r"./data/raw/insurance_company_modified.csv")
# Definir el directorio de salida para los datos limpios
#OUT_DIR  = Path(r"./data/clean_data")
# Crear el directorio de salida si no existe (incluyendo directorios padres)
#OUT_DIR.mkdir(parents=True, exist_ok=True)
# Definir la ruta completa del archivo CSV limpio de salida
#OUT_CSV    = OUT_DIR / "coil2000_clean.csv"
# Definir la ruta completa del archivo de reporte de tipos de datos de salida
#OUT_REPORT = OUT_DIR / "coil2000_dtypes_report.csv"


In [ ]:
# Definir los rangos válidos para algunas columnas numéricas para proceso de outliers
RANGES = {
    "MOSTYPE":  (1, 41),  # Rango para el tipo de cliente
    "MAANTHUI": (1, 10),  # Rango para el número de casas
    "MGEMOMV":  (1, 6),   # Rango para el ingreso promedio
    "MGEMLEEF": (1, 6),   # Rango para la edad promedio
    "MOSHOOFD": (1, 10),  # Rango para el tipo principal de cliente
    "MGODRK":   (0, 9),   # Rango para la afiliación religiosa
    "PWAPART":  (0, 9),   # Rango para el número de pólizas de seguro privado
    "AWAPART":  (1, 12),  # Rango para el número de pólizas de seguro de coche
    "CARAVAN":  (0, 1),   # Rango para si tiene caravana este es nuestra variable categórica (0: No, 1: Sí)
}

# Definir un patrón de expresión regular para identificar valores inválidos o nulos en strings
INVALID_PATTERN = re.compile(r"(?i)^\s*$|^(nan|none|null|n/a|invalid|\?|unknown|error|missing|-)$")

# 1) Función para parsear el diccionario de datos y obtener los nombres de las columnas
def parse_dictionary(txt_path: Path):
    # Leer el contenido del archivo de texto del diccionario
    txt = txt_path.read_text(encoding="utf-8", errors="ignore")
    # Lista para almacenar los nombres de las columnas
    names = []
    # Bandera para indicar si se está dentro de la sección de la tabla de columnas
    in_table = False
    # Iterar sobre cada línea del archivo
    for ln in txt.splitlines():
        # Eliminar espacios en blanco al inicio y final de la línea
        ln = ln.strip()
        # Verificar si la línea coincide con el inicio de la sección de la tabla
        if re.match(r"^1\s+\S+", ln):
            in_table = True
        # Si no se está dentro de la tabla, saltar a la siguiente línea
        if not in_table:
            continue
        # Verificar si la línea está vacía o empieza con "L0" (indicando el final de la tabla)
        if not ln or ln.startswith("L0"):
            break
        # Intentar extraer el número de columna y el nombre usando una expresión regular
        m = re.match(r"^(\d+)\s+(\S+)\s+.*$", ln)
        if m:
            # Agregar el número de columna y el nombre a la lista
            names.append((int(m.group(1)), m.group(2)))
    # Ordenar los nombres de las columnas por su número y obtener solo los nombres
    cols = [n for _, n in sorted(names, key=lambda x: x[0])]
    # Verificar si el número de columnas obtenidas coincide con el número esperado
    if len(cols) < N_COLS:
        raise ValueError(f"El diccionario tiene {len(cols)} nombres; se requieren {N_COLS}.")
    # Devolver solo el número de nombres de columnas
    return cols[:N_COLS]

# 2) Función para limpiar cadenas de caracters y reemplazarlos por NaN
def normalize_strings_to_nan(df: pd.DataFrame) -> pd.DataFrame:
    # Crear una copia del DataFrame para no modificar el original
    df = df.copy()
    # Iterar sobre las columnas de tipo 'object'
    for c in df.select_dtypes(include="object").columns:
        # Convertir la columna a tipo 'string' y eliminar espacios en blanco
        df[c] = df[c].astype("string").str.strip()
    # Reemplazar los valores que coinciden con el patrón de inválidos por NaN
    return df.replace(INVALID_PATTERN, np.nan, regex=True)

# 3) Función para castear los tipos de datos de las columnas
def cast_types(df: pd.DataFrame) -> pd.DataFrame:
    # Crear una copia del DataFrame
    df = df.copy()
    # Identificar las columnas que no son categóricas
    non_cat = [c for c in df.columns if c not in CAT_COLS]
    # Convertir las columnas no categóricas a tipo numérico, si hay errores NaN
    df[non_cat] = df[non_cat].apply(pd.to_numeric, errors="coerce")
    # Convertir las columnas especificadas como categóricas a tipo 'category'
    for c in CAT_COLS:
        # Verificar si la columna existe en el DataFrame
        if c in df.columns:
            df[c] = df[c].astype("category")
    # Devuelve el DataFrame con los tipos de datos casteados
    return df

# 4) Función para aplicar reglas de validación por intervalos cerrados
def apply_interval_rules(df: pd.DataFrame) -> pd.DataFrame:
    # Creamos una copia del DataFrame
    df = df.copy()
    # Imprimir la forma del DataFrame antes de aplicar las reglas
    print(df.shape)

    # Iterar sobre las columnas y sus rangos definidos
    for col, (lo, hi) in RANGES.items():
        # Verificar si la columna existe en el DataFrame
        if col not in df.columns:
            continue
        # Convertir la columna a tipo numérico
        s = pd.to_numeric(df[col].astype("string").str.strip(), errors="coerce") \
            if col in CAT_COLS or not pd.api.types.is_numeric_dtype(df[col]) else df[col]
        # Crear una máscara booleana para identificar los valores dentro del rango
        mask = s.between(lo, hi, inclusive="both")
        # Contar el número de filas que están fuera del rango
        removed = (~mask).sum()
        # Si se eliminaron filas, imprimir un mensaje
        if removed:
            print(f"{col} ∈ [{lo}, {hi}]: filas eliminadas = {removed}")
        # Filtrar el DataFrame para mantener solo las filas dentro del rango y crear una nueva copia
        df = df[mask].copy()
    # Si la columna "CARAVAN" existe y sus valores son 0 o 1, convertirla a tipo entero
    if "CARAVAN" in df.columns:
        # Obtener los valores únicos no nulos de la columna
        vals = set(pd.Series(df["CARAVAN"]).dropna().unique().tolist())
        # Verificar si los valores únicos están entre el rago {0, 1}
        if vals <= {0, 1}:
            # Convertir a numérico, luego a entero (permite NaN), y finalmente a entero nativo
            df["CARAVAN"] = pd.to_numeric(df["CARAVAN"], errors="coerce").astype("Int64").astype("int64")
    # Restablecer el índice del DataFrame después de la eliminación de filas
    df.reset_index(drop=True, inplace=True)
    # Devolver el DataFrame con las reglas de intervalo aplicadas
    return df

# 5) Función para eliminar filas con valores nulos
def enforce_no_nulls(df: pd.DataFrame) -> pd.DataFrame:
    # Reemplazar strings vacíos o que contienen solo espacios en blanco por NaN
    df = df.replace(r"^\s*$", np.nan, regex=True)
    # Guardar el número de filas antes de eliminar nulos
    before = len(df)
    # Eliminar las filas que contienen al menos un valor nulo y restablecer el índice
    df = df.dropna().reset_index(drop=True)
    # Imprimir el número de filas eliminadas debido a nulos
    print(f"Filas eliminadas por nulos/vacíos: {before - len(df)}")
    # Devuelve el DataFrame sin filas con nulos
    return df

# 6) Pipeline principal de limpieza y procesamiento de datos
def main():
    # Hace parse al diccionario para obtener los nombres de las columnas
    cols = parse_dictionary(DICT_TXT)

    # Leer el archivo CSV en un DataFrame, sin encabezado
    df = pd.read_csv(CSV_PATH, header=None)
    # Verificar si el número de columnas del CSV coincide con el número esperado
    if df.shape[1] < N_COLS:
        raise ValueError(f"El CSV tiene {df.shape[1]} columnas; se requieren {N_COLS}.")
    # Seleccionar solo el número esperado de columnas y crear una copia
    df = df.iloc[:, :N_COLS].copy()
    # Asignar los nombres de columnas obtenidos del diccionario
    df.columns = cols

    # Aplicar las funciones de limpieza en secuencia
    # Normalizar strings y reemplazar por NaN
    df = normalize_strings_to_nan(df)
    # Aplicar reglas de intervalo
    df = apply_interval_rules(df)
    # Hacer la conversión de tipos de datos
    df = cast_types(df)
    # Eliminar filas con nulos
    df = enforce_no_nulls(df)
    # Verificar por valores nulos en el DataFrame resultante
    assert df.isna().sum().sum() == 0

    # Guardar el DataFrame limpio en un archivo CSV
    df.to_csv(OUT_CSV, index=False)

    # Crear un reporte con información sobre las columnas del DataFrame limpio
    report = pd.DataFrame({
        # Nombres de las columnas
        "columna": df.columns,
        # Tipos de datos finales como strings
        "dtype_final": df.dtypes.astype(str).values,
        # Número de valores nulos por columna
        "nulos": df.isna().sum().values,
        # Número de valores únicos no nulos por columna
        "unicos": df.nunique(dropna=True).values,
        # Indica si la columna fue tratada como categórica
        "es_categorica": [c in CAT_COLS for c in df.columns],
    })
    # Guardar el reporte de tipos de datos en un archivo CSV
    report.to_csv(OUT_REPORT, index=False)

    # Imprimir mensajes de confirmación
    print(f"Guardado dataset limpio: {OUT_CSV}")
    print(f"Guardado reporte dtypes: {OUT_REPORT}")

# Ejecutar la función principal si el script se ejecuta directamente
if __name__ == "__main__":
    main()